<a href="https://colab.research.google.com/github/zhaocaiQ/GPT-3/blob/main/pytorch_lightning_acc_%EB%B0%A9%EB%B2%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#https://opendatascience.com/higher-level-pytorch-apis-a-short-introduction-to-pytorch-lightning/

In [2]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 700 kB 4.0 MB/s 
     |████████████████████████████████| 141 kB 51.1 MB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
     |████████████████████████████████| 5.8 MB 60.3 MB/s 
     |████████████████████████████████| 419 kB 59.0 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.9.1 which is inco

In [3]:
import pytorch_lightning as pl 
import torch  
import torch.nn as nn   
from torchmetrics import Accuracy  
class MultiLayerPerceptron(pl.LightningModule): 
    def __init__(self,image_shape=(1, 28, 28), hidden_units=(32, 16)): 
        super().__init__()      
        # new PL attributes: 
        self.train_acc = Accuracy() 
        self.valid_acc = Accuracy() 
        self.test_acc = Accuracy()     
        # Model similar to previous section: 
        input_size = image_shape[0] * image_shape[1] * image_shape[2] 
        all_layers = [nn.Flatten()] 
        for hidden_unit in hidden_units:  
            layer = nn.Linear(input_size, hidden_unit)  
            all_layers.append(layer)  
            all_layers.append(nn.ReLU())  
            input_size = hidden_unit    
        all_layers.append(nn.Linear(hidden_units[-1], 10))  
        all_layers.append(nn.Softmax(dim=1))  
        self.model = nn.Sequential(*all_layers)  
    def forward(self, x): 
        x = self.model(x) 
        return x 
    def training_step(self, batch, batch_idx): 
        x, y = batch 
        logits = self(x) 
        loss = nn.functional.cross_entropy(self(x), y) 
        preds = torch.argmax(logits, dim=1) 
        self.train_acc.update(preds, y) 
        self.log("train_loss", loss, prog_bar=True) 
        return loss 
    def training_epoch_end(self, outs): 
        self.log("train_acc", self.train_acc.compute())      
    def validation_step(self, batch, batch_idx): 
        x, y = batch 
        logits = self(x) 
        loss = nn.functional.cross_entropy(self(x), y) 
        preds = torch.argmax(logits, dim=1) 
        self.valid_acc.update(preds, y) 
        self.log("valid_loss", loss, prog_bar=True) 
        self.log("valid_acc", self.valid_acc.compute(), prog_bar=True) 
        return loss  
    def test_step(self, batch, batch_idx): 
        x, y = batch 
        logits = self(x) 
        loss = nn.functional.cross_entropy(self(x), y) 
        preds = torch.argmax(logits, dim=1) 
        self.test_acc.update(preds, y) 
        self.log("test_loss", loss, prog_bar=True) 
        self.log("test_acc", self.test_acc.compute(), prog_bar=True) 
        return loss  
    def configure_optimizers(self): 
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001) 
        return optimizer

In [4]:
from torch.utils.data import DataLoader 
from torch.utils.data import random_split 
from torchvision.datasets import MNIST 
from torchvision import transforms  
class MnistDataModule(pl.LightningDataModule): 
    def __init__(self, data_path='./'): 
        super().__init__() 
        self.data_path = data_path 
        self.transform = transforms.Compose([transforms.ToTensor()])          
    def prepare_data(self): 
        MNIST(root=self.data_path, download=True)   
    def setup(self, stage=None): 
        # stage is either 'fit', 'validate', 'test', or 'predict' 
        # here note relevant 
        mnist_all = MNIST(  
            root=self.data_path, 
            train=True, 
            transform=self.transform,   
            download=False 
        )   
        self.train, self.val = random_split( 
            mnist_all, [55000, 5000], generator=torch.Generator().manual_seed(1) 
        )  
        self.test = MNIST(  
            root=self.data_path, 
            train=False, 
            transform=self.transform,   
            download=False 
        )   
    def train_dataloader(self): 
        return DataLoader(self.train, batch_size=64, num_workers=4)  
    def val_dataloader(self): 
        return DataLoader(self.val, batch_size=64, num_workers=4)  
    def test_dataloader(self): 
        return DataLoader(self.test, batch_size=64, num_workers=4) 

In [5]:
torch.manual_seed(1)  
mnist_dm = MnistDataModule() 

In [6]:
mnistclassifier = MultiLayerPerceptron()  
if torch.cuda.is_available(): # if you have GPUs 
    trainer = pl.Trainer(max_epochs=10, gpus=1) 
else: 
    trainer = pl.Trainer(max_epochs=10)  
trainer.fit(model=mnistclassifier, datamodule=mnist_dm) 

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
